###  Earthquake location
In class you saw how the earthquakes mostly occur on tectonic plate boundaries. But how do we know that? we can use the travel time of seismic waves to estimate the epicentre of earthquakes. In today's lab we are going to locate an earthquake ourselves. First we need a network of seismometers. Today, we'll use network IU, which can be accessed through the FDSN client called "IRIS". Build an inventory of the stations of IU and plot the inventory. If you cannot remember this task, look through your previous labs.

Next, we will explain how we can estimate the travel time of different seismic phases (P, S, etc.) if we know (only) the epicentral distance. For that, we use our knowledge of the 1D seismic velocity model PREM.

#### The Primary Reference Earth Model (PREM)

In the Earth, the seismic wave speed is far from constant. More realistic P- and S-wave speeds as a function of the depth, based on seismic observations, are captured in the [Preliminary Reference Earth Model (PREM)](https://en.wikipedia.org/wiki/Preliminary_reference_Earth_model). The code cell below prints the columns of this model for the P-wave and the S-wave:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
url = "http://ds.iris.edu/files/products/emc/data/PREM/PREM_ANISOTROPIC.csv"
df = pd.read_csv(url, usecols=[0,2,3], names=['radius', 'vP','vS'], header=None)
vp = df['vP']/1000 # in km/s
vs = df['vS']/1000 # in km/s
r = df['radius']/1000 # radius in km
plt.plot(r,vp,label='$V_p$')
plt.plot(r,vs,label='$V_s$')
plt.xlabel('Distance to the centre of the Earth (km)')
plt.ylabel('Wave speed v (km/s)')
plt.legend()
plt.show()

### The main features of a radially symmetric Earth
You can divide the PREM model in four main regions. 

Discuss what you see in terms of these regions and relate these observations to what you know about the internal structure of the Earth?

#### Tracing your own seismic ray paths through the Earth

Based on Snell's Law, rays can be traced in the Earth. The code we use here is called [“TauP”](https://pubs.geoscienceworld.org/ssa/srl/article/70/2/154/142385/the-taup-toolkit-flexible-seismic-travel-time-and). Information about the package, including the manual can be downloaded [here](http://www.seis.sc.edu/TauP/). You could run TauP standalone, but [obspy](https://docs.obspy.org/) provides a Python class called
`Taup`, which comes with a function [plot_ray_paths](https://docs.obspy.org/packages/autogen/obspy.taup.tau.plot_ray_paths.html#obspy.taup.tau.plot_ray_paths). Read the docs on this function and plot the P-wave ray paths for an earthquake at 30 km deep for 30 paths.

Plot S- and P-wave arrivals from 0 to 360 degrees. Why are there no rays beyond approximately 100 degrees from the Earthquake? Comment here, or make a Markdown cell below your plot to write in.

Besides P- and S-waves in the mantle, there are a host of other — more complicated — seismic ray paths from earthquake to receiver. [This link](http://www.iris.edu/data/vocab.htm) describes the naming conventions.

What is a PKP phase?
Plot the PKP ray paths and P ray paths from 0–360 degrees. If we consider PKP a direct compressional arrival, compare what you see with what we discussed about P-wave arrivals and their range.

# **The travel times of seismic rays**

The previous tasks involved the path of seismic rays in the earth. But how about their travel times? One of the simplest examples of using obspy is to create a travel time plot of different seismic phases. Find the plot_travel_times function and plot the arrival times of S-waves between 0 and 100 degrees from an Earthquake at 30 km depth:

Make a single figure that contains the travel-times for P, S, SS, and PP phases between 0 and 100 degrees epicentral distance, for an earthquake at 30 km depth. Explain what each of these seismic phases mean: how and where did they travel?

#### Seismic data
Next, we'll download seismic data from an earthquake.
The following code downloads seismic data for station COR (Corvallis, Oregon) with the recordings of an Earthquake. It turns out, there is redundancy built into network IU and this station has three seismometers. We are looking at channel LHZ. Using the naming convention we showed you in previous weeks, find out what channel LHZ means in a markdown cell under the plot.

In [ ]:
from obspy.core import UTCDateTime

t1 = UTCDateTime('2025-07-29T23:20:00') # start time
t2 = t1 + 3600 # end time
st = client.get_waveforms("IU", "COR", "**", "LHZ", t1, t2)  # in later parts, you can ex-change SNZO for another stations.
st.plot();

Zoom in on the part of the graph that you think contains the important body wave arrivals. (If you use Colab, you cannot zoom in a figure panel, but you could use the [trim function](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.trim.html).) 

Try to identify the arrival time of the P, S, SS, and PP arrival. Your earlier work determines the order of these arrivals. 

By the way, the largest amplitudes in the seismogram are due to (slower) surface waves. Plot your estimates of arrivals on the graph by making use of this example code:

In [ ]:
example_arrival = t1 + 1800
fig = plt.figure()
st[2].trim(starttime=UTCDateTime("2025-07-29:23:30:00")).plot(fig=fig) # in this example there were three sensors, so I am only showing the third
ax = fig.gca()
ax.axvline(x=example_arrival.datetime, label='Arrival', color='blue')
plt.legend()
plt.show()

In [ ]:
# Write your code here

Estimate the delays between the arrivals of P, S, PP, and SS waves. Use these delay times and your graph of predicted arrival times from before to estimate the distance in degrees between station COR and the earthquake.


Below is a code to plot a circle with a radius equal to 1000 km from SNZO on a world map. Locate the lat and lon of station COR in your inventory, and plot a circle with a radius that equals the distance to the earthquake from COR. HINT: You will have to convert degrees to kilometres.

In [ ]:
import cartopy.crs as ccrs

# define a map, and its projection:
plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
ax.stock_img()

# Plot a circle on the map, centered on "station" with a lat lon. The radius is "radius"
station='SNZO'
radius = 1000 #km
lon= 174.704 
lat=-41.309

ax.tissot(rad_km=radius, lons=lon, lats=lat, alpha=0.2, color='r')
ax.text(lon,lat,station,transform=ccrs.Geodetic())
plt.show()

Obtain seismograms from two other stations for this time window, and estimate the epicentral distance for those two stations. Plot the circles.
If you did this well, your circles should intersect near the epicentre of this earthquake.

Next, estimate the origin time for this earthquake.

Find the earthquake in the USGS earthquake catalogue online. Add the epicentre of this earthquake to your map to see how close your estimate is to that of the professionals. You may have to make a new map that zooms in around the epicentre to judge and discuss your results. Name three reasons your circles do not intersect *exactly*:  

#### Bonus question
If you go back to your first travel time plot in this lab, it looks like there are arrival times overlapping. Plot the ray paths only for this section, and instead of a spherical plot, use a cartesian representation. Can you explain what you see there?